<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Aggregation" data-toc-modified-id="Aggregation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Aggregation</a></span><ul class="toc-item"><li><span><a href="#Lecture-objectives" data-toc-modified-id="Lecture-objectives-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Lecture objectives</a></span><ul class="toc-item"><li><span><a href="#Key-Takeaways" data-toc-modified-id="Key-Takeaways-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Key Takeaways</a></span></li></ul></li></ul></li></ul></div>

# Aggregation

## Lecture objectives

1. Demonstrate how to aggregate in `pandas` using `groupby`

Let's start off by loading the data that we used in the previous lecture, and joining two of the dataframes.

In [1]:
import pandas as pd
collisionDf = pd.read_csv('../data/Collisions.csv')
partyDf = pd.read_csv('../data/Parties.csv')
victimDf = pd.read_csv('../data/Victims.csv')
collisionDf.set_index('CASE_ID', inplace=True)
partyDf.set_index('CASE_ID', inplace=True)
joinedDf = collisionDf.join(partyDf, rsuffix='_from_party')
joinedDf.head()

,ACCIDENT_YEAR,PROC_DATE,JURIS,COLLISION_DATE,COLLISION_TIME,OFFICER_ID,REPORTING_DISTRICT,DAY_OF_WEEK,CHP_SHIFT,POPULATION,...,VEHICLE_YEAR,VEHICLE_MAKE,STWD_VEHICLE_TYPE,CHP_VEH_TYPE_TOWING,CHP_VEH_TYPE_TOWED,RACE,INATTENTION,SPECIAL_INFO_F,SPECIAL_INFO_G,ACCIDENT_YEAR_from_party
CASE_ID,,,,,,,,,,,,,,,,,,,,,
8167180,2018,2020-05-26,5600,2018-01-29,1700,3386,8125,1,5,5,...,NaN,-,N,60.0,NaN,H,NaN,-,-,2018
8167180,2018,2020-05-26,5600,2018-01-29,1700,3386,8125,1,5,5,...,2016.0,MAZDA,A,7.0,NaN,W,NaN,-,-,2018
8414912,2018,2018-02-14,5600,2018-01-30,1840,2740,9080,2,5,6,...,2017.0,HONDA,A,1.0,NaN,O,NaN,-,-,2018
8414912,2018,2018-02-14,5600,2018-01-30,1840,2740,9080,2,5,6,...,2001.0,FORD,D,22.0,NaN,W,NaN,-,-,2018
8414916,2018,2018-02-14,5600,2018-01-22,1425,2740,9122,1,5,6,...,2003.0,TOYOTA,D,22.0,NaN,W,NaN,-,-,2018


Many of the questions that we want to ask involve some form of aggregation, that creates a useful summary. And for this, `groupby` is the key. Basically, think what you want to aggregate over, and that's the field that you group by.

Example: what's the mean of killed and injured people by intoxication status? (Consult the [codebook](https://tims.berkeley.edu/help/SWITRS.php#Party_Level), sometimes called a data dictionary, for an explanation of what each code means.)

In [2]:
joinedDf.groupby('PARTY_SOBRIETY')[['NUMBER_KILLED','NUMBER_INJURED']].mean()

,NUMBER_KILLED,NUMBER_INJURED
PARTY_SOBRIETY,,
-,0.000000,1.333333
A,0.021429,1.462500
B,0.038462,1.346154
C,0.000000,1.000000
D,0.000000,1.000000
G,0.052632,1.263158
H,0.000000,1.375000


<div class="alert alert-block alert-info">
<strong>Warning:</strong> Are these accurate averages per collision? What might we need to do (conceptually)?
</div>

We have the correct mean for the average over parties. 

But remember that we did a one-to-many join earlier on. Some collisions are therefore duplicated in `joinedDf`, so the simple mean in effect weights collisions by the number of parties.

Sometimes, we need to transform the data first. Let's look at collisions that involve pedestrians.

In [3]:
print(joinedDf.groupby('PARTY_SOBRIETY').PEDESTRIAN_ACCIDENT.mean())

TypeError: unsupported operand type(s) for +: 'int' and 'str'

<div class="alert alert-block alert-info">
    <strong>Exercise:</strong> What went wrong? <em>Hint:</em> the first step is to look at the data field.
</div>

In [4]:
print(joinedDf.PEDESTRIAN_ACCIDENT.head())
print(joinedDf.PEDESTRIAN_ACCIDENT.unique())

CASE_ID
8167180      Y
8167180      Y
8414912    NaN
8414912    NaN
8414916    NaN
Name: PEDESTRIAN_ACCIDENT, dtype: object
['Y' nan]


It looks like we are trying to calculate the mean of a column that has `Y` and `NaN` values. `pandas` doesn't know how to do that.

We can fix this error by creating a new column, `ped_accident_boolean`, which is `True` if the value is `Y` and `False` otherwise. Then we can take the mean of the boolean column (`True` is considered a `1` and `False` a `0`).

In [5]:
joinedDf['ped_accident_boolean'] = joinedDf.PEDESTRIAN_ACCIDENT=='Y'
# let's look at the results
joinedDf[['PEDESTRIAN_ACCIDENT','ped_accident_boolean']].head()

,PEDESTRIAN_ACCIDENT,ped_accident_boolean
CASE_ID,,
8167180,Y,True
8167180,Y,True
8414912,NaN,False
8414912,NaN,False
8414916,NaN,False


This function would do exactly the same, and is easier to adapt if there are multiple values (e.g. `Y`, `N`, or missing) - you can just add more `elif` statements.

In [6]:
def convert_to_bool(ped_accident):
    if ped_accident=='Y':
        return True
    else:
        return False
joinedDf['ped_accident_boolean'] = joinedDf.PEDESTRIAN_ACCIDENT.apply(convert_to_bool)
joinedDf[['PEDESTRIAN_ACCIDENT','ped_accident_boolean']].head()

,PEDESTRIAN_ACCIDENT,ped_accident_boolean
CASE_ID,,
8167180,Y,True
8167180,Y,True
8414912,NaN,False
8414912,NaN,False
8414916,NaN,False


A more elegant and concise way is to use a `lambda` (anonymous) function.

Here, the value for `PEDESTRIAN_ACCIDENT` for that row is passed to the function as the variable `x`. Then that function returns `1` if x is `Y`, otherwise `0`.

In [7]:
joinedDf['ped_accident_boolean'] = joinedDf.PEDESTRIAN_ACCIDENT.apply(
                                        lambda x: True if x=='Y' else False)

joinedDf[['PEDESTRIAN_ACCIDENT','ped_accident_boolean']].head()

,PEDESTRIAN_ACCIDENT,ped_accident_boolean
CASE_ID,,
8167180,Y,True
8167180,Y,True
8414912,NaN,False
8414912,NaN,False
8414916,NaN,False


Now we can look at the results using `groupby`. For parties of different sobriety states, what are the proportions of pedestrian accidents? 

In [8]:
print(joinedDf.groupby('PARTY_SOBRIETY').ped_accident_boolean.mean())

PARTY_SOBRIETY
-    0.333333
A    0.069643
B    0.038462
C    0.000000
D    0.000000
G    0.210526
H    0.187500
Name: ped_accident_boolean, dtype: float64


It's not just means that you can generate with `groupby`. Standard deviations, counts, sums, and more are available. [See the documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html) for more examples of this very powerful function.

You can also group by the index, rather than a column, using `level=0`. This is useful if (say) your observations are indexed by census tract, and you want to calculate tract-level means or sums.

We can set `PARTY_SOBRIETY` as the index to illustrate this.

In [9]:
# size() gives the number of rows in each group
print(joinedDf.groupby('PARTY_SOBRIETY').ped_accident_boolean.size())

# identical results if we set the index to PARTY_SOBRIETY and group by the index
tmpDf = joinedDf.set_index('PARTY_SOBRIETY')
print(tmpDf.groupby(level=0).ped_accident_boolean.size())

# or if we do the above in a single line
print(joinedDf.set_index('PARTY_SOBRIETY').groupby(level=0).ped_accident_boolean.size())

PARTY_SOBRIETY
-      6
A    560
B     26
C      2
D      3
G     19
H     16
Name: ped_accident_boolean, dtype: int64
PARTY_SOBRIETY
-      6
A    560
B     26
C      2
D      3
G     19
H     16
Name: ped_accident_boolean, dtype: int64
PARTY_SOBRIETY
-      6
A    560
B     26
C      2
D      3
G     19
H     16
Name: ped_accident_boolean, dtype: int64


In [10]:
# sum gives the group-wise totals, i.e. the number of True values
print(joinedDf.groupby('PARTY_SOBRIETY').ped_accident_boolean.sum())

PARTY_SOBRIETY
-     2
A    39
B     1
C     0
D     0
G     4
H     3
Name: ped_accident_boolean, dtype: int64


<div class="alert alert-block alert-info">
<h3>Key Takeaways</h3>
<ul>
  <li>Group-wise means, totals, and sums are easy to compute using groupby.</li>
</ul>
</div>